## Setting of some variables and analysis parameters

* the name of the source 
* output directory for figures and high-level products

* the minimum S/N for LC adaptive binning (min_sn_lc)
* the minimum S/N for Time-Phase and Energy-Phase matricx adaptive binning (min_sn_matrix)
* number of bins for the folded pulsed profile (n_bins_pulse)
* start and stop frequency of frequenct search (in Hz)

* if nustar tasks will be run (useful for a fast rerun on processed data)
* if specific tasks of the notebook are to be run 

## log changes
2023-03-20
- introduced the possibility to rebin the matrixes from the last energy(time) channel
- statistic tests for PF

In [ ]:
#General 
source : str = '' # http://odahub.io/ontology#AstrophysicalObject
output_dir_figures_name : str = 'figures' # Output folder, it will be complemented by a time tag

#Timing
gbm_url : str = 'https://gammaray.nsstc.nasa.gov/gbm/science/pulsars/lightcurves/' #For the orbit
min_search_frequency : float = 1e-3  # minimum frequency to search for a peak in the LS periodogram 
max_search_frequency : float = 1e1   # maximum frequency to search for a peak in the LS periodogram
orbitfile : str = None #'orbit.dat' # None to avoid orbit correction
freq_search_interval :float = 0.02 # It will search epoch folding from (1-/+freq_search_interval) * LS_frequency 
nudot : float = 0.0
gti_base_name : str = 'user_gtis.fits' # The GTI user name
probability_linear_trend : float = 0.1 # The probability to detect a linear trend in phases
gti_filter_far : float = 5.7e-5 # False alarm rate for the light curve in GTI selection (5-sigma)
gti_in_level_down : float = 0 # no rate is taken below this value
gti_in_level_up : float = 10000 # no rate is taken above this value
gti_quantile : float = 0.9 # the quantile to pre-filter rate so that the Gaussian is detrmined withour extremes
lc_timebin : float = 1.0 # http://odahub.io/ontology#TimeBinSeconds

# Rebinning options and matrix setup
min_sn_lc : float = 30 # Minimum S/N for the light curve rebinning
min_sn_matrix : float = 8 # minimum signal-to-noise ratio for the energy-phase and time-phase matrixes
onlypulsedoption : bool = True # To use only the pulsed signal to set the minimum S/N
n_bins_pulse : list = [8, 16, 64, 32] # number of bins in the folded pulse profile
t_step_matrix : float = 1500  # time step of the time-phase matrix in sec
en_step_matrix : str = 'energybinning.dat'
#en_step_matrix : float = 1.0 # energy step of the energy-phase matrix in keV
e_min_tphase : float = 3.0 # minimum energy to accumulate the time-phase matrix
e_max_tphase : float = 30.0 # maximum energy to accumulate the time-phase matrix
    
### specific tasks which can be run in the notebook 
### useful to speed up reprocessing 
run_nustar_tasks :bool = False # generally this should be set to false for precomputed stuff
rerun_nustar_pipeline :bool = False # generally this should be set to false for precomputed stuff
compute_and_show_PF_methods : bool = False #To compute and show all PF methods
check_bayesian : bool = False  # If making the Bayesian block analysis for the variability
recompute_matrixes : bool = True # Force recomputing the matrices

In [ ]:
## Importing modules developed for this project:
# eg nustarpipeline process to wrap the nustar analysis for our needs, 
# nustarpipeline utils to collect useful functions
# pyxmmsas (originally for XMM) for spectral fitting

%matplotlib widget
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.stats import norm
import shutil
import os, sys
from astroquery.simbad import Simbad
from astropy import units as u
from astropy.coordinates import SkyCoord
from importlib import reload
from astropy.io import fits as pf
from nustarpipeline import process, utils
import pyxmmsas as pysas
import hratio
from scipy import interpolate
import json
from datetime import datetime
import logging
from time import gmtime, strftime

In [ ]:
mylocation=os.getcwd()
print('mylocation = ' + mylocation)
obsid=mylocation.split('/')[-1]
print('OBSID: '+ obsid)
if str(orbitfile) == 'None':
    print('Orbitfile is None')
    orbitfile = None

Setting local pfiles for heasoft

In [ ]:
if os.path.isdir('pfiles'):
    print('folder \'pfiles\' exist')
else:
    os.makedirs('pfiles')
pfiles = os.environ['PFILES'].split(';')
os.environ['PFILES']=mylocation+'/pfiles;'+pfiles[1]
os.environ['PFILES']

In [ ]:
# now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# output_dir_figures_name = output_dir_figures_name.split('/')[0]+'_'+now

if os.path.isdir(output_dir_figures_name):
    print('folder \'%s\' exist' % output_dir_figures_name)
else:
    os.makedirs(output_dir_figures_name)
    
output_dir_figures = mylocation+'/'+output_dir_figures_name

In [ ]:
file_handler = logging.FileHandler(filename='nustar_utils_%s.log' % (strftime("%Y-%m-%dT%H:%M:%S", gmtime())))
stdout_handler = logging.StreamHandler(sys.stdout)
handlers = [file_handler] #stdout_handler, 

logging.basicConfig(level=logging.INFO, format=' %(levelname)s - %(message)s', handlers=handlers)
logging.getLogger('').addHandler(logging.StreamHandler()) 

The obsid is derived from the folder structure

In [ ]:
reload(utils)

tmp = mylocation.split('/')

obsid=tmp[-1]
ra_src,dec_src = utils.get_target_coords_extern(source)
print(obsid)

## The following cell is used to re-extract energy selected lc and spectra (if run_nustar_tasks = True )

In [ ]:
reload(process)
if rerun_nustar_pipeline:
    process.wrap_process(obsid, ra_src, dec_src, 'obs', pipeline_flag=True, 
                            repository_location=os.environ['HOME']+'/NUSTAR/Repository')

# This displays a ds9 instance to define the region files 
* wait for ds9 to appear
* edit the regions for module A, save them as sourceA.reg and backgroundA.reg
* close the ds9 window
* wait for ds9 to appear
* edit the regions for module A, save them as sourceA.reg and backgroundA.reg
* close the ds9 window
* Note that edited region files should be stored under 
```
{{cwd}}/obs_spec
```
as 
```
sourceA.reg 
backgroundA.reg
sourceB.reg
backgroundB.reg
```

## Afterwards these regions will be used throughout the analysis

In [ ]:
if run_nustar_tasks:
    use_js9 = True
    try:
        import jpjs9
    except:
        use_js9 = False
    #reload(jpjs9)

    if use_js9:
        J_A = jpjs9.JS9(id='fpmAA')
        J_A.display()

In [ ]:
if run_nustar_tasks and use_js9:
    J_B = jpjs9.JS9(id='fpmB')
    J_B.display()

In [ ]:
if run_nustar_tasks:
    reload(process)

    process.wrap_process(obsid, ra_src, dec_src, 'obs', region_flag=True,  no_ds9_flag=True,
                     repository_location=os.environ['HOME']+'/NUSTAR/Repository'
                        )#, J_A=J_A, J_B=J_B)

In [ ]:
if run_nustar_tasks:
    
    process.wrap_process(obsid, ra_src, dec_src, 'obs', lc_flag=True, t_bin=lc_timebin, e_min=3.0, e_max=70.0,
                     repository_location=os.environ['HOME']+'/NUSTAR/Repository')
    process.wrap_process(obsid, ra_src, dec_src, 'obs', lc_flag=True, t_bin=lc_timebin, e_min=3.0, e_max=7.0,
                     repository_location=os.environ['HOME']+'/NUSTAR/Repository')
    process.wrap_process(obsid, ra_src, dec_src, 'obs', lc_flag=True, t_bin=lc_timebin, e_min=7.0, 
                     e_max=30.0, write_baryevtfile_flag=True, filter_evt_flag=True,
                     repository_location=os.environ['HOME']+'/NUSTAR/Repository')
    process.wrap_process(obsid, ra_src, dec_src, 'obs',  spec_flag=True, 
                         repository_location=os.environ['HOME']+'/NUSTAR/Repository')

In [ ]:
# for the next markdown
cwd = os.getcwd()

Rebin the light curve at minimum S/N to be defined by the user and identifies periods with variations of the hardness ratio

In [ ]:
#os.chdir('../')

reload(hratio)
lc_soft, lc_hard, h_ratio = hratio.hratio_func(mylocation+'/obs_lc/FPMA_3.0_7.0_sr.lc', 
                                               mylocation+'/obs_lc/FPMA_7.0_30.0_sr.lc', 'hratio.qdp', 
                                               min_sn_lc, flag_rebin=3)
#Gets rebinned LC
#input_rebinned_lc='lc_'+hr_file_name
t1=h_ratio._times_
dt1=h_ratio._dtimes_/2.
r1=lc_soft._rates_
e1=lc_soft._drates_
r2=lc_hard._rates_
e2=lc_hard._drates_
hr=h_ratio._rates_
e_hr=h_ratio._drates_
#t1,dt1,r1,e1,r2,e2,hr,e_hr=np.loadtxt(input_rebinned_lc, skiprows=14,unpack=True )
lc_rate=r1+r2
lc_error=np.sqrt(e1*2+e2**2)
time0=lc_soft._timezero_

In [ ]:
if check_bayesian:
    #This 95% purity
    ncp_prior = 1.32 + 0.577 *np.log10(len(r1)) + 7.6

    #add 10
    #add 0.7 as emprirical value to have quality 0.01
    s_n=np.mean(lc_rate/lc_error)
    s_n_hr=np.mean(hr/e_hr)
    N=len(lc_rate)
    print("N=%d"%(N), "S/N=%f"%(s_n), "S/N HR=%f"%(s_n_hr), "ncp_prior=%f"%(ncp_prior))
    mean_hr=hr.mean()
    mean_lc=lc_rate.mean()
    print("<hr>=%f"%(mean_hr), "<cr>=%f"%(mean_lc))

In [ ]:
if check_bayesian:
    from astropy.stats import bayesian_blocks
    ind=t1<=1e10
    n_attempts=50
    n_edges=np.zeros(n_attempts)
    for i in range(n_attempts):
            test_case=np.zeros(N)
            for j in range(N):
                test_case[j]=np.random.normal(loc=mean_hr, scale=e_hr[j], size=1)
    #test_case=np.random.normal(loc=s_n, scale=1, size=N)
    #edges = bayesian_blocks(t1[ind], hr[ind], e_hr[ind], fitness='easures', p0=.001)
            test_edges = bayesian_blocks(t1[ind], test_case[ind], e_hr[ind] , fitness='measures', ncp_prior=ncp_prior)
            n_edges[i]=len(test_edges)
    #print(i, len(test_edges)-2)
    print("On average, we have %.1f false positives, with values from %d to %d"%(n_edges.mean()-2, n_edges.min()-2, n_edges.max()-2))
    ind=t1<=1e11
    #edges = bayesian_blocks(t1[ind], hr[ind], e_hr[ind], fitness='measures', p0=.001)
    raw_edges = bayesian_blocks(t1[ind], hr[ind], e_hr[ind], fitness='measures', ncp_prior=ncp_prior)
    edges=raw_edges
    for i, edge in enumerate(edges):
        if i==0:
            edges[i]=t1[i]-dt1[i]
        elif i==len(edges)-1:
            edges[i]=t1[-1]+dt1[-1]
        else:
            idx=np.searchsorted(t1,edge,side='left') 
            edges[i]=t1[idx-1]+dt1[idx-1]
    #print(edge, edges[i])
    print('We have %d intervals switching between different HR values'%(len(edges)))

In [ ]:

ff = pf.open(lc_soft._filename_)
target=ff[1].header['OBJECT'].replace('_',' ').replace("p",'p')
date_obs=ff[1].header['DATE-OBS']
ff.close()
if check_bayesian:
    plt.rcParams['figure.figsize'] = [6, 6]
    ff,axes=plt.subplots(2,1, sharex=True)
    axes[0].errorbar(t1, hr, xerr=dt1, yerr=e_hr, linestyle='none')
    #axes=ff.get_axes()
    axes[0].vlines(edges,ymin=np.min(hr-e_hr),ymax=np.max(hr+e_hr), color='red')
    axes[0].set_ylabel('HR')
    axes[0].set_title('%s %s'%(target, obsid))
    axes[1].errorbar(t1, lc_rate, xerr=dt1, yerr=lc_error, linestyle='none')
    axes[1].set_xlabel('Time [s] from %s'%date_obs[0:21])
    axes[1].set_ylabel('Total rate')
    axes[1].vlines(edges,ymin=0,ymax=np.max(lc_rate+lc_error), color='red')
    ff.savefig(output_dir_figures+"/%s_%s_division_lc.pdf" % (source.replace(' ', '_'), obsid) )

plt.figure(figsize=(6,6))
plt.errorbar(hr, lc_rate, yerr=lc_error, xerr=e_hr, linestyle='none' )
plt.xlabel("Hardness")
plt.ylabel('Intensity')
plt.title('HID diagram %s %s'%(target, obsid))
plt.savefig(output_dir_figures+"/%s_%s_hid.pdf" % (source.replace(' ', '_'), obsid) )

In [ ]:
reload(process)

n_gti = process.create_user_gti(t1, lc_rate, lc_error, lognormal=True, gti_name=gti_base_name, 
                    in_level_down = gti_in_level_down, 
                    in_level_up = gti_in_level_up, 
                    quantile = gti_quantile, 
                    plot_file_name=output_dir_figures+'/%s_%s_GTI-filter.pdf' %(source.replace(' ', '_'), obsid), 
                    far=gti_filter_far, time0=time0, title=source + ' ' + obsid)
                   

if n_gti < 1:
    gti_name = None
else:
    gti_name = '../'+gti_base_name

## It finds the frequency of pulsation between a minimum and a maximum, use a narrow range to spare time
* check https://gammaray.nsstc.nasa.gov/gbm/science/ for a useful range of values
* If orbital correction needs to be done, please run timingsuite from the command line and enter the values, then use the orbit file as input
* if no orbital correction is needed, put `orbitfile=None` , which is the default
* _orbitfile will be used also to extract the matrixes, so it is defined as a variable_

This will try to retireve the orbitfile from the GBM page

In [ ]:
reload(utils)
if orbitfile is None or str(orbitfile) == 'None':
    print("Not making an orbit correction")
else:
    derived_orbitfile = utils.write_orbit_file_from_gbm_page(gbm_url, file_name='obs_lc/' + orbitfile)
    
    if derived_orbitfile is None:
        print("WARNING :: We could not read the orbit file, skipping orbit correction")
        orbitfile=None
    #     keep_orbit = input('We could not retrieve the orbit file from the GBM page, do you want to keep it ? (yes or no)')
    #     if 'y' in keep_orbit.lower():
    #         print("We will use %s orbit correction provided by user" % orbitfile)
    #     else:
    #         orbitfile=None
    

In [ ]:
reload(utils)
nu_start, nu_stop = utils.get_nu_search_limits(mylocation+'/obs_lc/FPMA_3.0_7.0_sr.lc', 
                                         freq_search_interval=freq_search_interval, 
                                         plot_filename=output_dir_figures+'/' + source.replace(' ', '_') + '_' + obsid +'_Lomb-Scargle.pdf',
                                         min_frequency=min_search_frequency, 
                                         max_frequency=max_search_frequency)

In [ ]:
reload(utils)
info_source = mylocation+'/obs_lc/ef_pipe_res.dat'
run_frequencysearch = not os.path.isfile(info_source)
#check wether the pulse period search has already be done in this run or before
#then choose wether to run the spin period search or not

os.chdir(mylocation+'/obs_lc/')
freq = utils.get_efold_frequency(nu_start, nu_stop, actual_search=run_frequencysearch, 
                         orbitfile=orbitfile)
os.chdir(mylocation)
print('Best frequency is %lf Hz or %lf s' % (freq, 1./freq) )

## TIME-PHASE matrix construction

In [ ]:
evt_pnt = pf.open(mylocation+'/obs_lc/sourceA.evt')
t_ref = evt_pnt[1].data['TIME'][0]
evt_pnt.close()
t_ref

In [ ]:
reload(utils)
os.chdir(mylocation+'/obs_lc')

check_time_matrix = os.path.isfile(mylocation+'/obs_lc/sourceA_TPHASE.fits') and (not recompute_matrixes)

In [ ]:
if check_time_matrix is False:
    utils.make_tphase(freq, n_bins=n_bins_pulse[-1], t_step=t_step_matrix, 
                  orbitfile=orbitfile, nudot=nudot, min_en=e_min_tphase, max_en=e_max_tphase, 
                  user_gti=gti_name, t_ref=t_ref)
else:
     print('Time-Phase matrix exists')   
os.chdir(mylocation)

## Reading and rebinning the TPHASE matrix at minimum SNR

In [ ]:
reload(utils)
os.chdir(mylocation+'/obs_lc')
t_min_orig, t_max_orig, pt_orig, dpt_orig, pt_orig_back, dpt_orig_back = utils.read_and_sum_matrixes('T', -1)
os.chdir(mylocation)

t_min, t_max, pt, dpt = utils.rebin_matrix(t_min_orig, t_max_orig, pt_orig, dpt_orig, min_sn_matrix, 
                                           only_pulsed=onlypulsedoption)
tt=(t_max+t_min)/2
dtt=(t_max-t_min)/2
tt_orig=(t_max_orig+t_min_orig)/2
dtt_orig=(t_max_orig-t_min_orig)/2

In [ ]:
#if run_frequencysearch:
reload(utils)

phases=[]
dphases=[]

for i in range(len(t_min)):
#     x= (t_max[i]+t_min[i])/2
#     dx= (t_max[i]-t_min[i])/2
    y=pt[i,:]
    dy=dpt[i,:]
    A, phi = utils.get_fourier_coeff(y)
    dA, dphi = utils.get_fourier_coeff_error(y, dy)
    phases.append(phi[0])
    dphases.append(dphi[0])
plt.figure(figsize=(6, 6*(5**.5 - 1)/2.))
cmap = plt.cm.magma
colors = iter(cmap(np.linspace(0, 1, 4)))
from cycler import cycler
plt.rc('axes', prop_cycle=(cycler('color', colors)))

phi0=utils.align_phases(np.array(phases))
dphi0=np.array(dphases)
#phi0[phi0<0.5]+=1
plt.errorbar(tt-tt[0], phi0, xerr=dtt, yerr=dphi0, linestyle='', marker='o')
plt.xlabel('Time')
plt.ylabel('$\phi_0$')
plt.title(source + ' ' + obsid)
prob, r, a, b = pysas.linear_fit_plot_bootstrap(tt-tt[0], phi0, dtt, dphi0, ax=plt.gca())
plt.savefig(output_dir_figures+"/%s_%s_time_vs_phaseshift.pdf" %(source.replace(' ', '_'), obsid))


## It removes a linear trend detected at more than 90% confidence level from the phase shift and refines the frequency.

In [ ]:

reload(utils)
os.chdir(mylocation+'/obs_lc')
n_iterations = 0
max_n_iterations = 2
while prob.max() < probability_linear_trend and n_iterations < max_n_iterations:
    freq = freq + a[1] 
    print('The frequency is %f Hz' % freq)
    utils.make_tphase(freq, n_bins=n_bins_pulse[-1], t_step=t_step_matrix, orbitfile=orbitfile, 
                      nudot=nudot, min_en=e_min_tphase, max_en=e_max_tphase, 
                      user_gti='../'+gti_name) 
    t_min_orig, t_max_orig, pt_orig, dpt_orig, pt_orig_back, dpt_orig_back = utils.read_and_sum_matrixes('T', -1)
    t_min,t_max, pt, dpt = utils.rebin_matrix(t_min_orig, t_max_orig, pt_orig, dpt_orig, 
                                              min_sn_matrix, t_ref=t_ref)
    tt=(t_max+t_min)/2
    dtt=(t_max-t_min)/2                                  
    phases=[]
    dphases=[]
    for i in range(len(t_min)):
        y=pt[i,:]
        dy=dpt[i,:]
        A, phi = utils.get_fourier_coeff(y)
        dA, dphi = utils.get_fourier_coeff_error(y, dy)
        phases.append(phi[0])
        dphases.append(dphi[0])
    plt.figure()
    phi0=utils.align_phases(np.array(phases))
    dphi0=np.array(dphases)
    plt.errorbar(tt-tt[0], phi0, xerr=dtt, yerr=dphi0, linestyle='', marker='o')
    plt.xlabel('Time')
    plt.ylabel('$\phi_0$')
    prob, r, a, b = pysas.linear_fit_plot_bootstrap(tt-tt[0], phi0, dtt, dphi0, ax=plt.gca())
    n_iterations += 1
os.chdir(mylocation)
if n_iterations> 0:
    print("We iterated %s times to get the phase correction " % n_iterations)

## It finds a frequency derivative if too many iterations (to be further tested)

In [ ]:
reload(pysas)
reload(utils)
os.chdir(mylocation+'/obs_lc')
if n_iterations >= max_n_iterations:
    plt.figure()
    plt.plot(tt - tt[0], phi0, marker='o', linestyle='')
    reload(pysas)
    prob, coeff = pysas.poly_fit_plot_bootstrap(tt-tt[0], phi0,  dtt, dphi0, deg=2, n_sample=1000, ax=plt.gca())  
    freq = freq - coeff[1][1]
    nudot = - 2* coeff[2][1]
    print('The frequency is %f Hz' % freq)
    utils.make_tphase(freq, n_bins=64, t_step=t_step_matrix, orbitfile=orbitfile, nudot=nudot, 
                      min_en=e_min_tphase, max_en=e_max_tphase, 
                      user_gti='../'+gti_name, t_ref=t_ref)   
    t_min_orig, t_max_orig, pt_orig, dpt_orig, pt_orig_back, dpt_orig_back = utils.read_and_sum_matrixes('T', -1)
    t_min,t_max, pt, dpt = utils.rebin_matrix(t_min_orig, t_max_orig, pt_orig, dpt_orig, min_sn_matrix)
    tt=(t_max+t_min)/2
    dtt=(t_max-t_min)/2                                  
    phases=[]
    dphases=[]
    for i in range(len(t_min)):
        y=pt[i,:]
        dy=dpt[i,:]
        A, phi = utils.get_fourier_coeff(y)
        dA, dphi= utils.get_fourier_coeff_error(y, dy)
        phases.append(phi[0])
        dphases.append(dphi[0])
    plt.figure()
    phi0=utils.align_phases(np.array(phases))
    dphi0=np.array(dphases)
    plt.errorbar(tt-tt[0], phi0, xerr=dtt, yerr=dphi0, linestyle='', marker='o')
    prob, r, a, b = pysas.linear_fit_plot_bootstrap(tt-tt[0], phi0, dtt, dphi0, ax=plt.gca())
    plt.xlabel('Time')
    plt.ylabel('$\phi_0$')
    plt.savefig(output_dir_figures+"/frequency_correction.pdf")
os.chdir(mylocation)

There are pulses with zeros, this causes the apparent phase lags, we do not consider them anymore

## Compute and plot the total pulse

In [ ]:
# pulse=np.sum(pt, 0)
# dpulse=np.sqrt(np.sum(dpt**2, 0))/pt.shape[0]

pulse = pt[0,:] * dtt[0]
dpulse = (dpt[0,:]**2) * dtt[0]
for i in range(1,len(dtt)):
    pulse+=pt[i,:] *dtt[i]
    dpulse+= (dpt[i,:] * dtt[i])**2

pulse/=np.sum(dtt)
dpulse=np.sqrt(dpulse)/np.sum(dtt)

plt.figure()
plt.errorbar(np.linspace(0,2,2*len(pulse)), np.tile(pulse,2), 
             xerr=0.5/len(pulse), yerr=np.tile(dpulse,2), 
             linestyle='-', marker='o')
plt.xlabel('Phase')
plt.ylabel('Rate (cts/s)')
plt.title('{} ObsID {}  N_bins={} Timebin={} s'.format(source,obsid,n_bins_pulse[-1],t_step_matrix ), loc='left')
plt.savefig(output_dir_figures+"/" +source.replace(' ', '_')+ '_' + obsid +  "_total_pulsed_profile.pdf")
print(len(pulse))


## Aligns the pulse to minimum of first harmonic

In [ ]:
reload(utils)
a, phi = utils.get_fourier_coeff(pulse)
print(phi)

In [ ]:
if check_time_matrix is False:
    t_ref -= (1.5 + phi[0]) / freq
    reload(utils)
    os.chdir(mylocation+'/obs_lc')
    utils.make_tphase(freq, n_bins=n_bins_pulse[-1], t_step=t_step_matrix, 
                      orbitfile=orbitfile, nudot=nudot, min_en=e_min_tphase, max_en=e_max_tphase, t_ref=t_ref,
                     user_gti=gti_name)
    os.chdir(mylocation)

## Rebin the aligned matrix

In [ ]:
reload(utils)
os.chdir(mylocation+'/obs_lc')
t_min_orig, t_max_orig, pt_orig, dpt_orig, pt_orig_back, dpt_orig_back  = utils.read_and_sum_matrixes('T', -1)
os.chdir(mylocation)
t_min, t_max, pt, dpt = utils.rebin_matrix(t_min_orig, t_max_orig, pt_orig, dpt_orig, min_sn_matrix, 
                                           only_pulsed=onlypulsedoption)
tt=(t_max+t_min)/2
dtt=(t_max-t_min)/2
tt_orig=(t_max_orig+t_min_orig)/2
dtt_orig=(t_max_orig-t_min_orig)/2

## Making and reading the Energy-Phase matrix
For all input n_bins !

In [ ]:
reload(utils)
recompute_matrixes=True
os.chdir(mylocation+'/obs_lc')
check_en_matrix = os.path.isfile(mylocation+'/obs_lc/sourceA_ENPHASE.fits') and (not recompute_matrixes)
check_en_matrix

In [ ]:
if check_en_matrix is False:
    for n_bins in n_bins_pulse:
        utils.make_enphase(freq, n_bins=n_bins, nudot=nudot, 
                   orbitfile=orbitfile, en_step='../../../'+en_step_matrix, 
                   t_ref=t_ref, user_gti=gti_name)
else: 
    print('Energy-Phase matrix exists')

os.chdir(mylocation)

## Rebinning the Energy-Phase matrix at min S/W

In [ ]:
reload(utils)
os.chdir(mylocation+'/obs_lc')
#e_min_orig, e_max_orig, pp_orig, dpp_orig, pp_orig_back, dpp_orig_back = utils.read_and_sum_matrixes('E', -1, use_counts=True, subtract_background=False)

e_min_orig, e_max_orig, pp_orig, dpp_orig, pp_orig_back, dpp_orig_back = utils.read_and_sum_matrixes('E', -1, 
                                                                        use_counts=True, 
                                                                        subtract_background=False)
ff=pf.open('sourceA.evt')
exposure=ff[1].header['EXPOSURE']
ontime = np.sum(ff[2].data['STOP']-ff[2].data['START'])
ff.close()
os.chdir(mylocation)
e_min, e_max, pp, dpp, pp_back, dpp_back = utils.rebin_matrix(e_min_orig, e_max_orig, pp_orig, dpp_orig, 
                                          min_sn_matrix,
                                          only_pulsed=True, use_counts=True, 
                                           background_matrix=(pp_orig_back, dpp_orig_back), flip = True)

print(exposure, ontime)
#e_min, e_max, pp, dpp, pp_back, dpp_back = utils.rebin_matrix(e_min_orig, e_max_orig, pp_orig, dpp_orig,  8, only_pulsed=True, use_counts=True, background=(pp_orig_back, dpp_orig_back))
#print(exposure, ontime)


In [ ]:
#utils.plot_matrix_as_lines(tee[90:91], pp_back[90:91], dpp_back[90:91], kind='E', offset=0., normalize=True)
#utils.plot_matrix_as_lines(tee[87:88], pp[87:88], dpp[87:88], kind='E', offset=0., normalize=True)

'''
utils.plot_matrix_as_lines(tee[-1], pp_back[-1], dpp_back[-1], kind='E', offset=0., normalize=True)
plt.title('{} ObsID {}  N_bins={}'.format(source,obsid,n_bins_pulse ), loc='left')
#plt.savefig(output_dir_figures+"energy_resolved_pprofiles.pdf")
'''

## Plotting the total pulse to check all is OK

In [ ]:
pulse=np.sum(pp, 0)#/pp.shape[0]
dpulse=np.sqrt(np.sum(dpp**2,0))#/pp.shape[0]
plt.figure()
plt.errorbar(np.linspace(0,2, 2*len(pulse)), np.tile(pulse, 2), 
             xerr=0.5/len(pulse), yerr=np.tile(dpulse, 2), linestyle='-', marker='o')
plt.xlabel('Phase')
plt.ylabel('Rate (cts/s)')
plt.title('{} ObsID {}  N_bins={}'.format(source,obsid,n_bins_pulse ), loc='left')

We plot the adaptive FFT for the last two bins using Poisson statistics (debug plots). It is not accurate as the number of harmonics is undefined


In [ ]:
if compute_and_show_PF_methods:
    reload(utils)

    ee_pulsed=(e_max+e_min)/2.
    dee_pulsed=(e_max-e_min)/2.

    pulsed_frac=np.zeros(len(e_min))
    dpulsed_frac=np.zeros(len(e_min))

    n_harm = np.zeros(len(e_min))

    dn_harm_min = np.zeros(len(e_min))
    dn_harm_max = np.zeros(len(e_min))

    methods = ['minmax', 'explicit_rms', 'adaptive', 'area', 'counts']

    n_harms=-1
    verbose=False
    use_poisson=True
    level = 5e-2

    all_pulsed_frac = np.zeros([len(ee_pulsed), len(methods)])
    all_pulsed_frac_err = np.zeros([len(ee_pulsed), len(methods)])
    s_n = np.zeros(len(ee_pulsed))

    for j, method_calc_rms in enumerate(methods):
    # print(method_calc_rms)
        for i in range(0, len(e_min)):
            plot=False
            x = pp[i,:]
            dx = dpp[i,:]
            x_b = pp_back[i,:]
            dx_b = dpp_back[i,:]

            if method_calc_rms=='minmax':
            #Min/MAX 
                pulsed_frac[i], dpulsed_frac[i] = utils.pulse_fraction_from_data_min_max(x,dx,background=x_b,
                                                                background_error=dx_b)
            elif method_calc_rms=='explicit_rms':
            #RMS with fixed n_harm
                pulsed_frac[i] = utils.pulse_fraction_from_data_rms(x,dx, n_harm=n_harms, background=x_b,
                                                                background_error=dx_b)
                dpulsed_frac[i] = utils.get_error_from_simul(x,dx, utils.pulse_fraction_from_data_rms,  n_simul=1000,
                                                            n_harm=n_harms, background=x_b,
                                                                background_error=dx_b)
            elif method_calc_rms == 'adaptive':
                #plt.figure()
            #RMS with adaptive n_harm
                if i <0:
                    plot=True
                pulsed_frac[i] = utils.fft_pulsed_fraction(x, dx, level=level, n_harm_min=2, n_harm_max=-1, 
                                                        plot=plot, 
                                                        verbose=verbose, background=x_b,
                                                                background_error=dx_b, statistics='cstat')
                dpulsed_frac[i] = utils.get_error_from_simul(x,dx, utils.fft_pulsed_fraction, level=level, 
                                                            n_harm_min=2, n_harm_max=-1, 
                                                            verbose=verbose,  n_simul=1000, plot=plot, 
                                                            use_poisson=use_poisson, background=x_b,
                                                                background_error=dx_b, statistics='cstat')
            elif method_calc_rms == 'area':
                pulsed_frac[i] = utils.pulsed_fraction_area(x,dx, background=x_b,
                                                                background_error=dx_b)
                dpulsed_frac[i] = utils.get_error_from_simul(x,dx,utils.pulsed_fraction_area, background=x_b,
                                                                background_error=dx_b)
            elif method_calc_rms == 'counts':
                pulsed_frac[i] = utils.pf_rms_counts(x,dx, background=x_b,
                                                                background_error=dx_b )
                dpulsed_frac[i] = utils.get_error_from_simul(x,dx,utils.pf_rms_counts, n_simul=1000, 
                                                            use_poisson=use_poisson, background=x_b,
                                                                background_error=dx_b)
            else:
                print('No method is defined')
            s_n[i] = np.sum(np.abs(x-np.mean(x)))/np.sqrt(np.sum(dx**2))
            out_str = '%d %.1f (%.1f %.1f) keV %.3f %.3f' %(i, s_n[i], 
                                                e_min[i], e_max[i],pulsed_frac[i], dpulsed_frac[i])

            if verbose:
                out_str+=' %d %d %d' %(n_harm[i], dn_harm_min[i], dn_harm_max[i])
            
            all_pulsed_frac[:,j] = pulsed_frac
            all_pulsed_frac_err[:,j] = dpulsed_frac

In [ ]:
if compute_and_show_PF_methods:
    plt.figure()

    markers=['x', 'd', '+', '.', 'o']
    for j, method_calc_rms in enumerate(methods):
        plt.errorbar(ee_pulsed,all_pulsed_frac[:,j],xerr=dee_pulsed,yerr=all_pulsed_frac_err[:,j],fmt=markers[j], 
                    linestyle='', alpha=0.5, label=method_calc_rms)

    plt.xscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('PF')
    plt.legend(loc='upper left')

In [ ]:
if compute_and_show_PF_methods:
    plt.figure()


    for j, method_calc_rms in enumerate(methods):
        plt.errorbar(ee_pulsed,all_pulsed_frac_err[:,j]/all_pulsed_frac[:,j],xerr=dee_pulsed,
                    fmt=markers[j], 
                    linestyle='', alpha=0.5, label=method_calc_rms)
    plt.errorbar(ee_pulsed,s_n/np.max(s_n)/10,xerr=dee_pulsed,
                    fmt='-', 
                    linestyle='', alpha=0.5, label='1/10 normalized S/N')

    plt.xscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('$\sigma_\mathrm{PF}$/PF')
    plt.legend()

In [ ]:
if compute_and_show_PF_methods:
    den = all_pulsed_frac[:,4]
    den_err = all_pulsed_frac_err[:,4] 

    ra_expl =  all_pulsed_frac[:,1] / den
    ra_expl_err =  ra_expl * np.sqrt( (all_pulsed_frac_err[:,1] / all_pulsed_frac[:,1])**2 +
                                    (den_err / den)**2 )
    ra_adap =  all_pulsed_frac[:,2] / den
    ra_adap_err =  ra_adap * np.sqrt( (all_pulsed_frac_err[:,2] / all_pulsed_frac[:,2])**2 +
                                    (den_err / den)**2 )

    plt.figure()
    plt.errorbar(ee_pulsed,ra_expl,xerr=dee_pulsed,yerr=ra_expl_err,
                    fmt='x', 
                    linestyle='', alpha=0.5, label='rms/counts')
    plt.errorbar(ee_pulsed,ra_adap,xerr=dee_pulsed,yerr=ra_adap_err,
                    fmt='+', 
                    linestyle='', alpha=0.5, label='fft/counts')
    plt.xscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Ratio')
    plt.legend()